In [4]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.utils import resample

In [5]:
df = pd.read_csv("aw_fb_data.csv")
df = df.drop(columns=["Unnamed: 0", "X1"])
le = preprocessing.LabelEncoder()
df["device"] = le.fit_transform(df["device"])

In [86]:
fitbit_df = df[df.device == 1].drop(columns=["device"])
fitbit_df = fitbit_df.sample(frac=1, random_state=42)
aw_df = df[df.device == 0].drop(columns=["device"])
aw_df = aw_df.sample(frac=1, random_state=42)
results = pd.DataFrame(columns=["Algorithm", "n_estimators", "eta", "max_depth", "n_neighbors", "AUC"])

## Experiments:

### Apple Watch Data Separately, Fitbit Data Separately, All Together:

1. XGBoost
    * n_estimators: 100, 200, 500, 1000
    * eta: 0.01, 0.1, 0.3
    * max_depth: 2, 3, 4, 5, 6
2. Decision Tree:
    * max_depth: 2, 5, 7, 10, 15, 20
3. Random Forest:
    * n_estimators: 100, 200, 500, 1000
    * max_depth: 2, 3, 4, 5, 6
4. K Nearest Neighbor:
    * n_neighbors: 3, 5, 7, 11
    

In [87]:
#DECISION TREE
max_depth = [2, 5, 7, 10, 15, 20]
for depth in max_depth:
    model = DecisionTreeClassifier(random_state = 42, max_depth = depth)
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    target = df.loc[:,"activity"]
    aucs=[]
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:] #changed to iloc for fitbit 
        test = df.loc[test_index,:]   #changed to iloc for fitbit 
        model.fit(train.iloc[:, :-1], train.iloc[:, -1])
        predictions = model.predict_proba(test.iloc[:, :-1])
        #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        aucs.append(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
    print("Average: " + str(sum(aucs)/len(aucs)))
    auc = str(sum(aucs)/len(aucs))
    results=results.append({"Algorithm": ["Decision Tree"], "n_estimators": [0], "eta": [0], "max_depth": [depth], "n_neighbors": [0], "AUC": [auc]}, ignore_index=True)


Average: 0.6245557253662671
Average: 0.7867763036119788
Average: 0.8364830279779388
Average: 0.8772386185203415
Average: 0.8819062487835302
Average: 0.8714341058042567


In [88]:
#RANDOM FOREST
n_estimators = [100, 200, 500, 1000]
max_depth =  [2, 3, 4, 5, 6]
for estimators in n_estimators:
    for depth in max_depth:
        model = RandomForestClassifier(random_state = 42, n_estimators = estimators, max_depth = depth)
        from sklearn.model_selection import StratifiedKFold
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        target = df.loc[:,"activity"]
        aucs=[]
        for train_index, test_index in skf.split(df, target):
            train = df.loc[train_index,:]
            test = df.loc[test_index,:]
            model.fit(train.iloc[:, :-1], train.iloc[:, -1])
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc = str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["Random Forest"], "n_estimators": [estimators], "eta": [0], "max_depth": [depth], "n_neighbors": [0], "AUC": [auc]}, ignore_index=True)


Average: 0.7590830631538752
Average: 0.8039899133100814
Average: 0.8379839908908824
Average: 0.8686687564776294
Average: 0.8947641068238333
Average: 0.7698801524004049
Average: 0.8083258640439
Average: 0.8438508257273089
Average: 0.8722846095923366
Average: 0.8979422480065071
Average: 0.7695939274113586
Average: 0.8098194019662077
Average: 0.8444208793191545
Average: 0.8724146148617369
Average: 0.8976786919605482
Average: 0.7708151196471851
Average: 0.8103949614910289
Average: 0.8443775716529599
Average: 0.8729186744257419
Average: 0.8981006723806939


In [89]:
#KNN
n_neighbors = [3, 5, 7, 11]
for neighbors in n_neighbors:
        model = KNeighborsClassifier(n_neighbors = neighbors)
        from sklearn.model_selection import StratifiedKFold
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        target = df.loc[:,"activity"]
        aucs=[]
        for train_index, test_index in skf.split(df, target):
            train = df.loc[train_index,:]
            test = df.loc[test_index,:]
            model.fit(train.iloc[:, :-1], train.iloc[:, -1])
            predictions = model.predict_proba(test.iloc[:, :-1])
            #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            aucs.append(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
        print("Average: " + str(sum(aucs)/len(aucs)))
        auc = str(sum(aucs)/len(aucs))
        results=results.append({"Algorithm": ["KNN"], "n_estimators": [0], "eta": [0], "max_depth": [0], "n_neighbors": [neighbors], "AUC": [auc]}, ignore_index=True)


Average: 0.8932573470758989
Average: 0.9064852820656256
Average: 0.9064657147921643
Average: 0.9016311310612508


In [91]:
#XGBoost
n_estimators = [100, 200, 500, 1000]
eta = [0.01, 0.1, 0.3]
max_depth = [2, 3, 4, 5, 6]
for eta in eta:
    for estimators in n_estimators:
        for depth in max_depth:
            model = XGBClassifier(random_state = 42, n_estimators = estimators, eta = eta, max_depth = depth)
            from sklearn.model_selection import StratifiedKFold
            skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
            target = df.loc[:,"activity"]
            aucs=[]
            for train_index, test_index in skf.split(df, target):
                train = df.loc[train_index,:]
                test = df.loc[test_index,:]
                model.fit(train.iloc[:, :-1], train.iloc[:, -1])
                predictions = model.predict_proba(test.iloc[:, :-1])
                #print(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
                aucs.append(roc_auc_score(test.iloc[:, -1], predictions, multi_class="ovr"))
            print("Average: " + str(sum(aucs)/len(aucs)))
            auc = str(sum(aucs)/len(aucs))
            results=results.append({"Algorithm": ["XGBoost"], "n_estimators": [estimators], "eta": [eta], "max_depth": [depth], "n_neighbors": [0], "AUC": [auc]}, ignore_index=True)


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:16:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:16:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[20:43:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:43:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:44:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Average: 0.9852962409201724


In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
plot_importance(rf)
pyplot.show()

In [92]:
results.to_csv("df_results.csv")